<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

This notebook should be run using with **Python 3.10** runtime environment. 

It requires service credentials for the following services:
  * Watson OpenScale
  
# OpenScale Headless Subscription

## Motivation
For security restrictions, or air gap deployment reasons, or firewall restrictions, some Customers are unwilling to expose their Machine Learning model scoring endpoint for external applications like OpenScale for monitoring purposes. The other use is in case of Batch Scoring models, where the scoring happens asynchronously, OpenScale does not have access to the scoring end point. But customers are looking for measuring the performance of their models by logging the payload data and feedback data to OpenScale data mart and thereby configure and evaluate the OpenScale monitors against this data.

## Solution
The solution is, with OpenScale, the customers can create a custom ML provider with an empty deployment URL, and there by configure an headless subscription by describing the payload data, followed by logging the payload data and configuring the monitors.

## Workflow
The notebook will configure OpenScale with a Headless subscription, where it showcases the following aspects related to Batch Scoring Models:

* Get hold of the training data and create a Pandas dataframe against it.
* Using the Training Statistics Generation API from OpenScale SDK to generate the training statistics.
* Create an OpenScale Headless susbcription by including the generated training statistics.
* Get hold of the output of the batch scoring model, and transform it in the format that OpenScale payload logging API understands.
     * This is the important step for customers who are using batch scoring models and wants to use OpenScale for monitoring the scored data.
     * The motivation here is, the batch scoring can happen in any scoring environment. All the OpenScale needs is, fetch the scored data, transform it such that OpenScale understands and log it.
* Log the payload data by calling the OpenScale DataSets API.
* Configure Explain Monitor, and generate an explanation on a transaction that is randomly selected from the logged payload data.
* Configure Fairness Monitor, and perform bias checking on the logged payload data.
* Log scored label data for performing quality monitor analysis.
* Configure Quality Monitor and run the quality monitor evaluation.
* Generate the Drift Archive using the training data, following by configuring the Drift monitor
* Run Drift evaluation.

# Setup <a name="setup"></a>

## Package installation

In [ ]:
!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1
!pip install --upgrade "ibm-watson-openscale~=3.0.34" --no-cache | tail -n 1
!pip install --upgrade "ibm-metrics-plugin[notebook]~=3.0.9" --no-cache | tail -n 1

### Action: restart the kernel!

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Configure credentials

In [2]:
CLOUD_API_KEY = "<Incldue your user API Key here>"
IAM_URL="https://iam.ng.bluemix.net/oidc/token"

In [3]:
#masked
WML_CREDENTIALS = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": CLOUD_API_KEY
}

Here, this notebook uses a credit risk model deployment in WML. This can be replaced with the scoring engine of your choice, but make sure the scoring response is in the format that OpenScale understands for monitor processing.

In [4]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.360'

In [5]:
wml_client.spaces.list(limit=10)

------------------------------------  -------------------------------------------------------------------  ------------------------
ID                                    NAME                                                                 CREATED
4021f1d9-c203-4e9f-97f6-4766dd48155b  prod-space                                                           2024-08-05T04:42:04.665Z
be45ab4c-1fb7-440c-9b03-2909067e45e0  Automotive Demo - Quality report summarization                       2024-05-27T13:13:27.233Z
e0ee6250-7ef6-42c3-8ffa-350d9b0df578  pre-prod-space                                                       2024-02-28T07:35:30.368Z
63c5982f-7160-41c4-86f7-1310a8ab32cb  prompt-space                                                         2024-01-15T19:29:21.535Z
f04e0e73-a1b7-4ae9-a08d-7e16add4fe08  llm. space                                                           2023-11-24T13:57:05.167Z
d1afbea3-e899-4ed3-b9a6-0686751508c3  wml                                                    

,ID,NAME,CREATED
0,4021f1d9-c203-4e9f-97f6-4766dd48155b,prod-space,2024-08-05T04:42:04.665Z
1,be45ab4c-1fb7-440c-9b03-2909067e45e0,Automotive Demo - Quality report summarization,2024-05-27T13:13:27.233Z
2,e0ee6250-7ef6-42c3-8ffa-350d9b0df578,pre-prod-space,2024-02-28T07:35:30.368Z
3,63c5982f-7160-41c4-86f7-1310a8ab32cb,prompt-space,2024-01-15T19:29:21.535Z
4,f04e0e73-a1b7-4ae9-a08d-7e16add4fe08,llm. space,2023-11-24T13:57:05.167Z
5,d1afbea3-e899-4ed3-b9a6-0686751508c3,wml,2023-09-21T07:06:42.577Z
6,6f7c3969-6d3f-4f9a-b97a-b534f4e4fef3,AutoAIDemo,2023-08-25T02:50:27.113Z
7,0b7992c2-3991-4145-a5ba-d5b428261171,openscale-express-path-preprod-80e6093f-5acf-4...,2023-08-16T06:59:18.538Z
8,3226c381-5ae0-4bc4-b306-fc638c785e47,openscale-express-path-80e6093f-5acf-4eb7-9da6...,2023-08-16T06:58:57.332Z


In [6]:
space_uid = '<Include the space id>'
wml_client.set.default_space(space_uid)

'SUCCESS'

In [7]:
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    deployment_uid = deployment["metadata"]["id"]
    deployment_name = deployment["entity"]["name"]
    print(deployment_name + ' : ' + deployment_uid)

drugs_multiclass_deployment : 0d84d926-8ef6-477f-acf6-ff06d77daf82
Text Binary Classifier deployment : 26af6dfb-8745-4ba8-a00c-403cd2d77cb4
hmr : 39153a06-4127-4053-97be-766fb433f786
Scikit German Risk Deployment WML V4 : 437fcbd9-bec2-4421-b672-c4f8464d7a60
Text Binary Classifier deployment : 82270e4a-2e1d-43c1-9ddd-dda3ad7faea6
Amazon Mobile Review Classifier deployment2 : 8edfc920-a4bc-47cd-83fd-9e9674476dec
Spark German Risk Deployment - Training stats : d2a8d768-7c54-49fe-a964-0e364c209540
Amazon Mobile Review Classifier deployment2 : ff674bc4-3070-41b1-ab48-ffff938bffb5


In [8]:
deployment_uid = '<Include the deployment id>'

### Get the training data

In [9]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv

--2024-08-08 08:45:17--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689622 (673K) [text/plain]
Saving to: ‘german_credit_data_biased_training.csv’

german_credit_data_ 100%[===================>] 673.46K  --.-KB/s    in 0.1s    

2024-08-08 08:45:17 (6.90 MB/s) - ‘german_credit_data_biased_training.csv’ saved [689622/689622]



In [10]:
import pandas as pd
data_df=pd.read_csv ("german_credit_data_biased_training.csv")
data_df.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk


In [11]:
feature_columns=["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
cat_features=["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
class_label = "Risk"
prediction_column = "predictedLabel"
probability_column = "probability"

# Generate the common configuration archive

In [12]:
VERSION = "6.0.0"

common_parameters = {
    "problem_type" : "binary", 
    "label_column": class_label,
    "probability_column": probability_column,
    "prediction_column": prediction_column,
    "feature_columns": feature_columns,
    "categorical_columns": cat_features,
    "enable_quality": False,
    "enable_fairness": False,
    "enable_explainability": True,
    "enable_drift_v2": True,
    "enable_drift": False,
    "notebook_version": VERSION,
    "input_data_type": "structured",
}

fairness_parameters = {}
drift_v2_parameters = {
    "advanced_controls": {
        "enable_model_quality_drift": False
    }
}

# Uncomment the below if you want to generate LIME global explanation
# Score function is mandatory to be specified to generate the LIME global explanation
'''explainability_parameters = {
    "global_explanation": {
        "enabled": True,
        "sample_size": 50, # the sample size of the records to be considered for computing payload global explanation
        "explanation_method": "lime" 
    }
}'''

explainability_parameters = {}

score = None

## [Optional] Score Function

This is required if you are configuring :
- Explainability (for computing LIME global explanation)

Please update the score function which will be used to generate explainability archive. 

The output of the score function should be a 2 arrays :
1. Array of probabilities
2. Array of model prediction


Please note:
- User is expected to make sure that the data type of the "class label" column selected and the prediction column are same. For eg : If class label is numeric, the prediction array should also be numeric
- Each entry of a probability array should have all the probabilities of the unique class label .
  For eg: If the model_type=multiclass and unique class labels are A, B, C, D . Each entry in the probability array should be a array of size 4 . Eg : [ [0.50,0.30,0.10,0.10] ,[0.40,0.20,0.30,0.10]...]
- **Please update the score function below**

In [13]:
def score(training_data_frame):
      
    #The data type of the label column and prediction column should be same .
    #User needs to make sure that label column and prediction column array should have the same unique class labels
    
    feature_columns = list(training_data_frame.columns)
    if class_label in feature_columns:
        feature_columns.remove(class_label)
    training_data_rows = training_data_frame[feature_columns].values.tolist()
    
    payload_scoring = {
      wml_client.deployments.ScoringMetaNames.INPUT_DATA: [{
           "fields": feature_columns,
           "values": [x for x in training_data_rows]
      }]
    }

    score = wml_client.deployments.score(deployment_uid, payload_scoring)
    score_predictions = score.get('predictions')[0]

    prob_col_index = list(score_predictions.get('fields')).index(probability_column)
    predict_col_index = list(score_predictions.get('fields')).index(prediction_column)

    if prob_col_index < 0 or predict_col_index < 0:
        raise Exception("Missing prediction/probability column in the scoring response")

    import numpy as np
    probability_array = np.array([value[prob_col_index] for value in score_predictions.get('values')])
    prediction_vector = np.array([value[predict_col_index] for value in score_predictions.get('values')])

    return probability_array, prediction_vector

In [14]:
from ibm_watson_openscale.utils.configuration_utility import ConfigurationUtility

config_util = ConfigurationUtility(
    training_data=data_df,
    common_parameters=common_parameters,
    scoring_fn=score)

config_util.create_configuration_package(
    explainability_parameters=explainability_parameters if "explainability_parameters" in locals() else None,
    drift_v2_parameters=drift_v2_parameters if "drift_v2_parameters" in locals() else None,
    fairness_parameters=fairness_parameters if "fairness_parameters" in locals() else None,
    display_link=True)

Training Statistics generated.
Explain Archive generated.
Computing feature importance for Drift v2 using lime global explanation.
Feature importance computed
Drift v2 Archive generated.


Download the common configuration package using the link provided above. In the next cell, we will unpack the common configuration archive and extract the files from it. 

In [15]:
import tarfile

# Download "configuration_archive.tar.gz" from project to local directory and 
# extract the artifacts
# from ibm_watson_studio_lib import access_project_or_space
# wslib = access_project_or_space()
# wslib.download_file("configuration_archive.tar.gz")

archive_file_path = "configuration_archive.tar.gz"
with tarfile.open(archive_file_path, "r:gz") as tar:
    tar.extractall('.')
    
# On unpacking the configuration archive, the files [common_configuration.json, drift_v2_archive.tar.gz, 
# explainability.tar.gz] will be extracted to the current directory. 
# Output may differ depending on which monitors were enabled while running the 
# common configuration notebook

### This JSON contains the training statistics

In [16]:
config_json = config_util.training_stats
config_json

{'common_configuration': {'problem_type': 'binary',
  'label_column': 'Risk',
  'prediction': 'predictedLabel',
  'probability': ['probability'],
  'input_data_schema': {'type': 'struct',
   'fields': [{'name': 'CheckingStatus',
     'type': 'string',
     'nullable': True,
     'metadata': {}},
    {'name': 'LoanDuration', 'type': 'long', 'nullable': True, 'metadata': {}},
    {'name': 'CreditHistory',
     'type': 'string',
     'nullable': True,
     'metadata': {}},
    {'name': 'LoanPurpose',
     'type': 'string',
     'nullable': True,
     'metadata': {}},
    {'name': 'LoanAmount', 'type': 'long', 'nullable': True, 'metadata': {}},
    {'name': 'ExistingSavings',
     'type': 'string',
     'nullable': True,
     'metadata': {}},
    {'name': 'EmploymentDuration',
     'type': 'string',
     'nullable': True,
     'metadata': {}},
    {'name': 'InstallmentPercent',
     'type': 'long',
     'nullable': True,
     'metadata': {}},
    {'name': 'Sex', 'type': 'string', 'nullable

# Configure OpenScale 

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [17]:
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.utils import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

import json
import requests
import base64
from requests.auth import HTTPBasicAuth
import time

## Get a instance of the OpenScale SDK client

In [18]:
authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY, disable_ssl_verification=True)
wos_client = APIClient(authenticator=authenticator)
wos_client.version

'3.0.39'

## OpenScale DataMart

Watson OpenScale uses a database to store payload and feedback logs and calculated metrics. Here we are using already configured data mart in IBM Cloud.

In [19]:
wos_client.data_marts.show()

None,None,True,active,2024-08-06 06:02:36.001000+00:00,80e6093f-5acf-4eb7-9da6-7ba9bf56a929


In [20]:
data_marts = wos_client.data_marts.list().result.data_marts
data_mart_id=data_marts[0].metadata.id
print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart 80e6093f-5acf-4eb7-9da6-7ba9bf56a929


In [21]:
data_mart_details = wos_client.data_marts.list().result.data_marts[0]
data_mart_details.to_dict()

{'metadata': {'id': '80e6093f-5acf-4eb7-9da6-7ba9bf56a929',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:data_mart:80e6093f-5acf-4eb7-9da6-7ba9bf56a929',
  'url': '/v2/data_marts/80e6093f-5acf-4eb7-9da6-7ba9bf56a929',
  'created_at': '2024-08-06T06:02:36.001000Z',
  'created_by': 'IBMid-662005298W',
  'modified_at': '2024-08-06T06:02:37.673000Z',
  'modified_by': 'IBMid-662005298W'},
 'entity': {'service_instance_crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929::',
  'internal_database': True,
  'database_configuration': {'database_type': 'postgresql',
   'credentials': {'secret_id': '7ddff6b7-8100-45ea-8f84-562181f87179'},
   'location': {'schema_name': '80e6093f-5acf-4eb7-9da6-7ba9bf56a929'}},
  'status': {'state': 'active'}}}

In [22]:
wos_client.service_providers.show()

## Remove existing service provider

Multiple service providers for the same engine instance are avaiable in Watson OpenScale. To avoid multiple service providers of used WML instance in the tutorial notebook the following code deletes existing service provder(s) and then adds new one.

In [23]:
SERVICE_PROVIDER_NAME = "OpenScale Headless Service Provider"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook to showcase Headless Subscription functionality."

In [24]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

## Add service provider

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model.

Note: Here the service provider is created with empty credentials, meaning no endpoint. Just to demonstrate the use case were we don't need an actual end point serving requests.

In [25]:
MLCredentials = {}
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        operational_space_id = "production",
        credentials=MLCredentials,
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider 2eb29272-1f94-43b7-83f3-4b080224e9d6 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [26]:
print(wos_client.service_providers.get(service_provider_id).result)

{
  "metadata": {
    "id": "2eb29272-1f94-43b7-83f3-4b080224e9d6",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:service_provider:2eb29272-1f94-43b7-83f3-4b080224e9d6",
    "url": "/v2/service_providers/2eb29272-1f94-43b7-83f3-4b080224e9d6",
    "created_at": "2024-08-08T03:16:07.531000Z",
    "created_by": "IBMid-662005298W"
  },
  "entity": {
    "name": "OpenScale Headless Service Provider",
    "service_type": "custom_machine_learning",
    "credentials": {
      "secret_id": "b2fa6151-ef1e-4755-ab84-41f518244aeb"
    },
    "operational_space_id": "production",
    "status": {
      "state": "active"
    }
  }
}


## Subscriptions

Remove existing credit risk subscriptions

This code removes previous subscriptions to the model to refresh the monitors with the new model and new data.

In [27]:
wos_client.subscriptions.show()

## Remove the existing subscription

In [28]:
SUBSCRIPTION_NAME = "GCR Headless Subscription"

In [29]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    if subscription.entity.asset.name == '[asset] ' + SUBSCRIPTION_NAME:
        sub_model_id = subscription.metadata.id
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [30]:
print("Data Mart ID: " + data_mart_id)
print("Service Provide ID: " + service_provider_id)
import uuid
asset_id = str(uuid.uuid4())
asset_name = '[asset] ' + SUBSCRIPTION_NAME
url = None

asset_deployment_id = str(uuid.uuid4())
asset_deployment_name = asset_name

Data Mart ID: 80e6093f-5acf-4eb7-9da6-7ba9bf56a929
Service Provide ID: 2eb29272-1f94-43b7-83f3-4b080224e9d6


In [31]:
prediction_column = prediction_column
probability_columns = [probability_column]
predicted_target_column = prediction_column
subscription_details = wos_client.subscriptions.add(data_mart_id,
    service_provider_id,
    asset=Asset(
        asset_id=asset_id,
        name=asset_name,
        url=url,
        asset_type=AssetTypes.MODEL,
        input_data_type=InputDataType.STRUCTURED,
        problem_type=ProblemType.BINARY_CLASSIFICATION
    ),
    deployment=None,    
    training_data_stats=config_json,
    prediction_field = prediction_column,
    predicted_target_field = predicted_target_column,
    probability_fields = probability_columns,
    background_mode = False,
    deployment_name = asset_name
    ).result

subscription_id = subscription_details.metadata.id
print("Subscription id {}".format(subscription_id))

Creating headless subscription as both deployment id and deployment space id are null



 Waiting for end of adding subscription fdaead88-3199-4c53-8a7f-85117919f62c 




active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------


Subscription id fdaead88-3199-4c53-8a7f-85117919f62c


In [32]:
wos_client.subscriptions.get(subscription_id).result.to_dict()

{'metadata': {'id': 'fdaead88-3199-4c53-8a7f-85117919f62c',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:subscription:fdaead88-3199-4c53-8a7f-85117919f62c',
  'url': '/v2/subscriptions/fdaead88-3199-4c53-8a7f-85117919f62c',
  'created_at': '2024-08-08T03:16:30.288000Z',
  'created_by': 'IBMid-662005298W',
  'modified_at': '2024-08-08T03:16:34.137000Z',
  'modified_by': 'IBMid-662005298W'},
 'entity': {'data_mart_id': '80e6093f-5acf-4eb7-9da6-7ba9bf56a929',
  'service_provider_id': '2eb29272-1f94-43b7-83f3-4b080224e9d6',
  'asset': {'asset_id': '88baf778-f949-453c-ac82-e3b452d8a33b',
   'url': 'https://dummyModelUrl',
   'name': '[asset] GCR Headless Subscription',
   'asset_type': 'model',
   'problem_type': 'binary',
   'input_data_type': 'structured'},
  'asset_properties': {'input_data_schema': {'type': 'struct',
    'fields': [{'metadata': {'measure': 'discrete',
       'modeling_role': 'feature'},
    

### The following code fetches the data set id, against which we would be performing the payload logging

In [33]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

Payload data set id: c5629ae5-1176-4e83-bf3b-c76c7f20bf8a


## Push a payload record to setup the required schemas in the subscription

This is the location where one needs to fetch the output of the batch scoring model and construct the payload as per the OpenScale Payload Logging format.

Note : No scoring is done against the model. The PayloadRecord is constructed with the request and response from the model/deployment.

## Scoring Request Payload

In [34]:
scoring_request =   {
        "fields": [
            "CheckingStatus",
            "LoanDuration",
            "CreditHistory",
            "LoanPurpose",
            "LoanAmount",
            "ExistingSavings",
            "EmploymentDuration",
            "InstallmentPercent",
            "Sex",
            "OthersOnLoan",
            "CurrentResidenceDuration",
            "OwnsProperty",
            "Age",
            "InstallmentPlans",
            "Housing",
            "ExistingCreditsCount",
            "Job",
            "Dependents",
            "Telephone",
            "ForeignWorker",
            "Risk"
        ],
        "values": [
            [
                "no_checking",
                28,
                "outstanding_credit",
                "appliances",
                5990,
                "500_to_1000",
                "greater_7",
                5,
                "male",
                "co-applicant",
                3,
                "car_other",
                55,
                "none",
                "free",
                2,
                "skilled",
                2,
                "yes",
                "yes",
                "Risk"
            ],
            [
                "greater_200",
                22,
                "all_credits_paid_back",
                "car_used",
                3376,
                "less_100",
                "less_1",
                3,
                "female",
                "none",
                2,
                "car_other",
                32,
                "none",
                "own",
                1,
                "skilled",
                1,
                "none",
                "yes",
                "No Risk"
            ],
            [
                "no_checking",
                39,
                "credits_paid_to_date",
                "vacation",
                6434,
                "unknown",
                "greater_7",
                5,
                "male",
                "none",
                4,
                "car_other",
                39,
                "none",
                "own",
                2,
                "skilled",
                2,
                "yes",
                "yes",
                "Risk"
            ],
            [
                "0_to_200",
                20,
                "credits_paid_to_date",
                "furniture",
                2442,
                "less_100",
                "unemployed",
                3,
                "female",
                "none",
                1,
                "real_estate",
                42,
                "none",
                "own",
                1,
                "skilled",
                1,
                "none",
                "yes",
                "No Risk"
            ],
            [
                "greater_200",
                4,
                "all_credits_paid_back",
                "education",
                4206,
                "less_100",
                "unemployed",
                1,
                "female",
                "none",
                3,
                "savings_insurance",
                27,
                "none",
                "own",
                1,
                "management_self-employed",
                1,
                "none",
                "yes",
                "No Risk"
            ],
            [
                "greater_200",
                23,
                "credits_paid_to_date",
                "car_used",
                2963,
                "greater_1000",
                "greater_7",
                4,
                "male",
                "none",
                4,
                "car_other",
                46,
                "none",
                "own",
                2,
                "skilled",
                1,
                "none",
                "yes",
                "Risk"
            ],
            [
                "no_checking",
                31,
                "prior_payments_delayed",
                "vacation",
                2673,
                "500_to_1000",
                "1_to_4",
                3,
                "male",
                "none",
                2,
                "real_estate",
                35,
                "stores",
                "rent",
                1,
                "skilled",
                2,
                "none",
                "yes",
                "Risk"
            ],
            [
                "no_checking",
                37,
                "prior_payments_delayed",
                "other",
                6971,
                "500_to_1000",
                "1_to_4",
                3,
                "male",
                "none",
                3,
                "savings_insurance",
                54,
                "none",
                "own",
                2,
                "skilled",
                1,
                "yes",
                "yes",
                "Risk"
            ],
            [
                "no_checking",
                14,
                "all_credits_paid_back",
                "car_new",
                1525,
                "500_to_1000",
                "4_to_7",
                3,
                "male",
                "none",
                4,
                "real_estate",
                33,
                "none",
                "own",
                1,
                "skilled",
                1,
                "none",
                "yes",
                "No Risk"
            ],
            [
                "less_0",
                10,
                "prior_payments_delayed",
                "furniture",
                4037,
                "less_100",
                "4_to_7",
                3,
                "male",
                "none",
                3,
                "savings_insurance",
                31,
                "none",
                "rent",
                1,
                "skilled",
                1,
                "none",
                "yes",
                "Risk"
            ]
        ]
    }

## Scoring Response Payload

In [35]:
scoring_response = {
    "predictions": [
        {
            "fields": [
                "prediction",
                "probability"
            ],
            "values": [
                [
                    "Risk",
                    [
                        0.104642951112211,
                        0.895357048887789
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.892112895920181,
                        0.10788710407981907
                    ]
                ],
                [
                    "Risk",
                    [
                        0.4863177905287259,
                        0.5136822094712741
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.980811537315731,
                        0.01918846268426898
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.9053052561083984,
                        0.09469474389160164
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.5315146773053994,
                        0.4684853226946007
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.7689466209701616,
                        0.23105337902983833
                    ]
                ],
                [
                    "Risk",
                    [
                        0.41317664143643873,
                        0.5868233585635613
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.9190247585206522,
                        0.08097524147934775
                    ]
                ],
                [
                    "No Risk",
                    [
                        0.781841942776921,
                        0.21815805722307902
                    ]
                ]
            ]
        }
    ]
}

### Construct the payload using the scoring_request and scoring_response and then log the records

In [36]:
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

records_list=[]
for x in range(10):
    pl_record = PayloadRecord(request=scoring_request, response=scoring_response)
    records_list.append(pl_record)

wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=records_list)

### Make sure the records reached the payload logging table inside the OpenScale DataMart.

In [37]:
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    raise Exception("Payload logging did not happen!")

Number of records in the payload logging table: 44


## Fetch the subscription details to confirm output data schemas are setup

In [38]:
wos_client.subscriptions.get(subscription_id).result.to_dict()

{'metadata': {'id': 'fdaead88-3199-4c53-8a7f-85117919f62c',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:subscription:fdaead88-3199-4c53-8a7f-85117919f62c',
  'url': '/v2/subscriptions/fdaead88-3199-4c53-8a7f-85117919f62c',
  'created_at': '2024-08-08T03:16:30.288000Z',
  'created_by': 'IBMid-662005298W',
  'modified_at': '2024-08-08T03:16:34.137000Z',
  'modified_by': 'IBMid-662005298W'},
 'entity': {'data_mart_id': '80e6093f-5acf-4eb7-9da6-7ba9bf56a929',
  'service_provider_id': '2eb29272-1f94-43b7-83f3-4b080224e9d6',
  'asset': {'asset_id': '88baf778-f949-453c-ac82-e3b452d8a33b',
   'url': 'https://dummyModelUrl',
   'name': '[asset] GCR Headless Subscription',
   'asset_type': 'model',
   'problem_type': 'binary',
   'input_data_type': 'structured'},
  'asset_properties': {'training_data_schema': {'type': 'struct',
    'fields': [{'metadata': {'columnInfo': {'columnLength': 64},
       'measure': 'discr

# Explainability Monitor Configuration
From the notebook, perform offline scoring against the customer model, create an explain archive and save this archive to data mart.

* Only Local explanations and Lime global explanations are supported.
* For contrastive explanations, as scoring is needed and because it is headless subscription without any deployment URL, contrastive explanations are not supported.

## [Optional] Specify the Explainability Configuration

Provide the following explainability configuration to enable LIME global explanation. 
Set explainability parameters to None if LIME global explanation is not required.

 - global_explanation: The global explanation parameters.
     - enabled: Enable the global explanation.
     - sample_size: The sample size of the records to be considered for computing global explanation in the payload window.
 - lime: The lime explanation parameters
     - perturbations_count: The no of perturbations created when generating a local explanation
 - local_explanation_method: The default local explanation method to be used when generating local explanation.

In [39]:
'''explainability_parameters = {
    "global_explanation": {
        "enabled": True,
        "sample_size": 50, # the sample size of the records to be considered for computing payload global explanation
        "explanation_method": "lime" 
    }
}'''
explainability_parameters = {}

### Upload the explain archive generated as part of the common configuration archive to OpenScale data mart

In [40]:
# Uncomment the below code if you are running the notebook from Watson Studio
'''from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

wslib.download_file("explainability.tar.gz")'''

# Applicable only when global explanation is enabled

with open("explainability.tar.gz", mode="rb") as perturbations_tar:
    wos_client.monitor_instances.upload_explainability_archive(subscription_id=subscription_id, archive=perturbations_tar)

print("Uploaded perturbations scoring response archive successfully.")

Uploaded perturbations scoring response archive successfully.


## Enable the Explainability monitor

In [41]:
print("Creating monitor instances...")

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    # Please note that lime global explanation is only supported from CPD 4.6.4
     "global_explanation":{
        "enabled": True
     },
    "training_statistics": config_json.get("explainability_configuration")
}

explain_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explain_monitor_instance_id = explain_monitor_details.metadata.id
print(explain_monitor_details)

Creating monitor instances...
{
  "metadata": {
    "id": "00b67c0b-4af6-4da6-a23e-c6df16c2f97e",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:monitor_instance:00b67c0b-4af6-4da6-a23e-c6df16c2f97e",
    "url": "/v2/monitor_instances/00b67c0b-4af6-4da6-a23e-c6df16c2f97e",
    "created_at": "2024-08-08T03:28:03.335000Z",
    "created_by": "IBMid-662005298W"
  },
  "entity": {
    "data_mart_id": "80e6093f-5acf-4eb7-9da6-7ba9bf56a929",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "fdaead88-3199-4c53-8a7f-85117919f62c"
    },
    "parameters": {
      "global_explanation": {
        "enabled": true
      },
      "training_statistics": {
        "mins": {
          "12": 19,
          "4": 250,
          "15": 1,
          "10": 1,
          "1": 4,
          "17": 1,
          "7": 1
        },
        "categorical_columns": [
        

## Check monitor instance status

In [42]:
explain_status = None
from datetime import datetime
while explain_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explain_monitor_instance_id).result
    explain_status = monitor_instance_details.entity.status.state
    if explain_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), explain_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), explain_status)


03:28:54 active


## Run an on-demand evaluation.
### This is applicable only when lime global explanation is enabled.

In [43]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=explain_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "6102d8a3-3d07-4918-9c0f-6b6047a209c5",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:6102d8a3-3d07-4918-9c0f-6b6047a209c5",
    "url": "/v2/monitor_instances/7c8c6942-0607-4849-9450-4209ea5b7adc/runs/6102d8a3-3d07-4918-9c0f-6b6047a209c5",
    "created_at": "2024-07-02T08:26:49.232000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "config_modified_at": "2024-07-02T08:26:25.895753Z",
      "config_package_file": "explainability.tar.gz",
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "lime": {
        "enabled": true,
        "features_count": 10,
        "perturbations_count": 208
      },
      "validate_table_job_app_id": null,
      "validate_table_job_id": "2db53ca8-dc6c-476f-baf3-48e1998ce19d",
      "validate_table_job_output_path": "explainability_configuration/d0cf6915-6897-4afb-a08c-36f1af1c186d/

In [44]:
# Check run status

explainability_run_status = None
while explainability_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=explain_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    explainability_run_status = monitoring_run_details.entity.status.state
    if explainability_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
        time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)

08:26:56 running
08:27:56 running
08:28:56 running
08:29:56 running
08:30:56 running
08:31:57 running
08:32:57 finished


## Display Explainability metrics

In [45]:
# Applicable only when global explanation is enabled
wos_client.monitor_instances.show_metrics(monitor_instance_id=explain_monitor_instance_id)

### Trigger a local explanation.

In [46]:
payload_data = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id,output_type='pandas').result
explanation_types = ["lime"]

scoring_ids = payload_data.head(1)['scoring_id'].tolist()
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types, subscription_id=subscription_id).result

explanation_task_ids=result.metadata.explanation_task_ids
explanation_task_ids

['d3a3aac2-7bd6-4b75-bfac-62c89d711728']

### Wait for the local explanation to complete.

In [47]:
def finish_explanation_tasks(sample_size = 1):
    finished_explanations = []
    finished_explanation_task_ids = []
    
    # Check for the explanation task status for finished status. 
    # If it is in-progress state, then sleep for some time and check again. 
    # Perform the same for couple of times, so that all tasks get into finished state.
    for i in range(0, 5):
        # for each explanation
        print('iteration ' + str(i))
        
        #check status for all explanation tasks
        for explanation_task_id in explanation_task_ids:
            if explanation_task_id not in finished_explanation_task_ids:
                result = wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id, subscription_id=subscription_id ).result
                print(explanation_task_id + ' : ' + result.entity.status.state)
                if (result.entity.status.state == 'finished' or result.entity.status.state == 'error') and explanation_task_id not in finished_explanation_task_ids:
                    finished_explanation_task_ids.append(explanation_task_id)
                    finished_explanations.append(result)


        # if there is altest one explanation task that is not yet completed, then sleep for sometime, 
        # and check for all those tasks, for which explanation is not yet completeed.
        
        if len(finished_explanation_task_ids) != sample_size:
            print('sleeping for some time..')
            time.sleep(10)
        else:
            break
                    
    return finished_explanations

### Find the explain task status

In [48]:
finished_explanations = finish_explanation_tasks(1)

iteration 0
d0430ad8-31f9-4088-a2a2-85bb93073d47 : in_progress
37912fdb-8b1e-49bc-b90c-78c18f489bdc : in_progress
sleeping for some time..
iteration 1
d0430ad8-31f9-4088-a2a2-85bb93073d47 : in_progress
37912fdb-8b1e-49bc-b90c-78c18f489bdc : in_progress
sleeping for some time..
iteration 2
d0430ad8-31f9-4088-a2a2-85bb93073d47 : finished
37912fdb-8b1e-49bc-b90c-78c18f489bdc : finished


## Print explain task output

In [49]:
for result in finished_explanations:
    print(result)

iteration 0
d0430ad8-31f9-4088-a2a2-85bb93073d47 : in_progress
37912fdb-8b1e-49bc-b90c-78c18f489bdc : in_progress
sleeping for some time..
iteration 1
d0430ad8-31f9-4088-a2a2-85bb93073d47 : in_progress
37912fdb-8b1e-49bc-b90c-78c18f489bdc : in_progress
sleeping for some time..
iteration 2
d0430ad8-31f9-4088-a2a2-85bb93073d47 : finished
37912fdb-8b1e-49bc-b90c-78c18f489bdc : finished


# Fairness configuration

The code below configures fairness monitoring for our model. It turns on monitoring for two features, sex and age. In each case, we must specify:
    
Which model feature to monitor One or more majority groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes One or more minority groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 80%) Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 100 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

### Create Fairness Monitor Instance

In [50]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
thresholds = [
                {
                    "metric_id": "statistical_parity_difference",
                    "type": "lower_limit",
                    "value": -0.15
                },
                {
                    "metric_id": "statistical_parity_difference",
                    "type": "upper_limit",
                    "value": 0.15
                },
                {
                    "metric_id": "average_odds_difference",
                    "type": "lower_limit",
                    "value": -0.15
                },
                {
                    "metric_id": "average_odds_difference",
                    "type": "upper_limit",
                    "value": 0.15
                },
                {
                    "metric_id": "average_abs_odds_difference",
                    "type": "lower_limit",
                    "value": -0.15
                },
                {
                    "metric_id": "average_abs_odds_difference",
                    "type": "upper_limit",
                    "value": 0.15
                },
                {
                    "metric_id": "false_negative_rate_difference",
                    "type": "lower_limit",
                    "value": -0.15
                },
                {
                    "metric_id": "false_negative_rate_difference",
                    "type": "upper_limit",
                    "value": 0.15
                },
                {
                    "metric_id": "false_positive_rate_difference",
                    "type": "lower_limit",
                    "value": -0.15
                },
                {
                    "metric_id": "false_positive_rate_difference",
                    "type": "upper_limit",
                    "value": 0.15
                },
                {
                    "metric_id": "false_discovery_rate_difference",
                    "type": "lower_limit",
                    "value": -0.15
                },
                {
                    "metric_id": "false_discovery_rate_difference",
                    "type": "upper_limit",
                    "value": 0.15
                },
                {
                    "metric_id": "error_rate_difference",
                    "type": "lower_limit",
                    "value": -0.15
                },
                {
                    "metric_id": "error_rate_difference",
                    "type": "upper_limit",
                    "value": 0.15
                },
                {
                    "metric_id": "false_omission_rate_difference",
                    "type": "lower_limit",
                    "value": -0.15
                },
                {
                    "metric_id": "false_omission_rate_difference",
                    "type": "upper_limit",
                    "value": 0.15
                }
            ]
parameters = {
    "features": [
        {
            "feature": "Sex",
            "majority": [
                "male"
            ],
            "metric_ids": [
                "statistical_parity_difference",
                "average_odds_difference",
                "average_abs_odds_difference",
                "false_negative_rate_difference",
                "false_positive_rate_difference",
                "false_discovery_rate_difference",
                "error_rate_difference",
                "false_omission_rate_difference",
                "fairness_value"
            ],
            "minority": [
                "female"
            ],
            "threshold": 0.98
        },
        {
            "feature": "Age",
            "majority": [
                [
                    25,
                    100
                ]
            ],
            "metric_ids": [
                "statistical_parity_difference",
                "average_odds_difference",
                "average_abs_odds_difference",
                "false_negative_rate_difference",
                "false_positive_rate_difference",
                "false_discovery_rate_difference",
                "error_rate_difference",
                "false_omission_rate_difference",
                "fairness_value"
            ],
            "minority": [
                [
                    18,
                    24
                ]
            ],
            "threshold": 0.98
        }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 100
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result




 Waiting for end of monitor instance creation bb4f8f7b-b059-4eb3-8614-102a8f47a1bb 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [51]:
fairness_monitor_instance_id = fairness_monitor_details.metadata.id
fairness_monitor_instance_id

'bb4f8f7b-b059-4eb3-8614-102a8f47a1bb'

### Get Fairness Monitor Instance

In [52]:
wos_client.monitor_instances.show()

80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,fdaead88-3199-4c53-8a7f-85117919f62c,subscription,fairness,2024-08-08 03:37:15.241000+00:00,bb4f8f7b-b059-4eb3-8614-102a8f47a1bb
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,fdaead88-3199-4c53-8a7f-85117919f62c,subscription,explainability,2024-08-08 03:28:03.335000+00:00,00b67c0b-4af6-4da6-a23e-c6df16c2f97e
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,pending,fdaead88-3199-4c53-8a7f-85117919f62c,subscription,drift_v2,2024-08-08 03:19:17.629000+00:00,7c3cba10-46ed-4218-b8c3-173d58df5b9c
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,fdaead88-3199-4c53-8a7f-85117919f62c,subscription,model_health,2024-08-08 03:16:32.811000+00:00,c2f435f1-3f59-4c9c-966d-bdc917b05ccc
80e6093f-5acf-4eb7-9da6-7ba9bf56a929,active,fdaead88-3199-4c53-8a7f-85117919f62c,subscription,performance,2024-08-08 03:16:34.280000+00:00,ec97e1a3-4569-4145-a37e-cf5e15109675


### Get run details
In case of production subscription, initial monitoring run is triggered internally. Checking its status

In [53]:
runs = wos_client.monitor_instances.list_runs(fairness_monitor_instance_id, limit=1).result.to_dict()
fairness_monitoring_run_id = runs["runs"][0]["metadata"]["id"]
run_status = None
while(run_status not in ["finished", "error"]):
    run_details = wos_client.monitor_instances.get_run_details(fairness_monitor_instance_id, fairness_monitoring_run_id).result.to_dict()
    run_status = run_details["entity"]["status"]["state"]
    print('run_status: ', run_status)
    if run_status in ["finished", "error"]:
        break
    time.sleep(10)

run_status:  finished


### Fairness run output

In [54]:
wos_client.monitor_instances.get_run_details(fairness_monitor_instance_id, fairness_monitoring_run_id).result.to_dict()

{'metadata': {'id': '78d3ff5d-d0f2-4bfb-b68b-a011f920f9e3',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/7e6e9ae0ab81bd329f59dc94206b3c07:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:run:78d3ff5d-d0f2-4bfb-b68b-a011f920f9e3',
  'url': '/v2/monitor_instances/bb4f8f7b-b059-4eb3-8614-102a8f47a1bb/runs/78d3ff5d-d0f2-4bfb-b68b-a011f920f9e3',
  'created_at': '2024-08-08T03:37:17.454000Z',
  'created_by': 'iam-ServiceId-667418e4-d5c6-4160-ad84-6f669f7bf88c'},
 'entity': {'triggered_by': 'user',
  'parameters': {'is_group_bias_completed': True,
   'measurement_id': 'b10dd665-8915-4684-9ed6-128e0f6a37f6',
   'total_records_processed': 0},
  'status': {'state': 'finished',
   'queued_at': '2024-08-08T03:37:17.443000Z',
   'started_at': '2024-08-08T03:37:19.741000Z',
   'updated_at': '2024-08-08T03:37:22.285000Z',
   'completed_at': '2024-08-08T03:37:22.176000Z',
   'message': 'bias run is successful.',
   'failure': {'trace': '6b99e4d7-9e4a-4279-a3e5-b400eed7479c',
    'errors': [{'code': 'AIQ

In [55]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2024-08-08 03:37:21.501900+00:00,fairness_value,b10dd665-8915-4684-9ed6-128e0f6a37f6,100.0,None,None,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,bb4f8f7b-b059-4eb3-8614-102a8f47a1bb,78d3ff5d-d0f2-4bfb-b68b-a011f920f9e3,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:37:21.501900+00:00,fairness_value,b10dd665-8915-4684-9ed6-128e0f6a37f6,100.0,None,None,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-24']",fairness,bb4f8f7b-b059-4eb3-8614-102a8f47a1bb,78d3ff5d-d0f2-4bfb-b68b-a011f920f9e3,subscription,fdaead88-3199-4c53-8a7f-85117919f62c


# Quality monitoring and feedback logging

## Enable quality monitoring

The code below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 70%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [56]:
import time

#time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result




 Waiting for end of monitor instance creation 56e970b2-7bf2-405c-9f7b-2b026f6012ea 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [57]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

'56e970b2-7bf2-405c-9f7b-2b026f6012ea'

## Get feedback logging dataset ID

In [58]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

In [59]:
feedback_dataset_id

'0cdf405e-08b4-4c15-9b7d-2cd4c05a717a'

In [60]:
feedback_payload = {
    "fields": [
        "CheckingStatus",
        "LoanDuration",
        "CreditHistory",
        "LoanPurpose",
        "LoanAmount",
        "ExistingSavings",
        "EmploymentDuration",
        "InstallmentPercent",
        "Sex",
        "OthersOnLoan",
        "CurrentResidenceDuration",
        "OwnsProperty",
        "Age",
        "InstallmentPlans",
        "Housing",
        "ExistingCreditsCount",
        "Job",
        "Dependents",
        "Telephone",
        "ForeignWorker",
        "Risk",
        "_original_probability",
        "_original_prediction",
        "_debiased_probability",
        "_debiased_prediction"        
    ],
    "values": [
        [
            "less_0",
            18,
            "credits_paid_to_date",
            "car_new",
            462,
            "less_100",
            "1_to_4",
            2,
            "female",
            "none",
            2,
            "savings_insurance",
            37,
            "stores",
            "own",
            2,
            "skilled",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "less_0",
            15,
            "prior_payments_delayed",
            "furniture",
            250,
            "less_100",
            "1_to_4",
            2,
            "male",
            "none",
            3,
            "real_estate",
            28,
            "none",
            "own",
            2,
            "skilled",
            1,
            "yes",
            "no",
            "No Risk",
            [
                0.7419002139563244,
                0.25809978604367556
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "0_to_200",
            28,
            "credits_paid_to_date",
            "retraining",
            3693,
            "less_100",
            "greater_7",
            3,
            "male",
            "none",
            2,
            "savings_insurance",
            32,
            "none",
            "own",
            1,
            "skilled",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.6935080115729353,
                0.3064919884270647
            ],
            "Risk",
            [
                0.8,
                0.2
            ],
            "Risk"
        ],
        [
            "no_checking",
            28,
            "prior_payments_delayed",
            "education",
            6235,
            "500_to_1000",
            "greater_7",
            3,
            "male",
            "none",
            3,
            "unknown",
            57,
            "none",
            "own",
            2,
            "skilled",
            1,
            "none",
            "yes",
            "Risk",
            [
                0.331110352092386,
                0.668889647907614
            ],
            "Risk",
            [
                0.9,
                0.1
            ],
            "Risk"
        ],
        [
            "no_checking",
            32,
            "outstanding_credit",
            "vacation",
            9604,
            "500_to_1000",
            "greater_7",
            6,
            "male",
            "co-applicant",
            5,
            "unknown",
            57,
            "none",
            "free",
            2,
            "skilled",
            2,
            "yes",
            "yes",
            "Risk",
            [
                0.11270206970758759,
                0.8872979302924124
            ],
            "Risk",
            [
                0.1,
                0.9
            ],
            "Risk"
        ],
        [
            "no_checking",
            9,
            "prior_payments_delayed",
            "car_new",
            1032,
            "100_to_500",
            "4_to_7",
            3,
            "male",
            "none",
            4,
            "savings_insurance",
            41,
            "none",
            "own",
            1,
            "management_self-employed",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.6704819620865308,
                0.32951803791346923
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "less_0",
            16,
            "credits_paid_to_date",
            "vacation",
            3109,
            "less_100",
            "4_to_7",
            3,
            "female",
            "none",
            1,
            "car_other",
            36,
            "none",
            "own",
            2,
            "skilled",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.6735810290914039,
                0.3264189709085961
            ],
            "Risk",
            [
                0.6,
                0.4
            ],
            "Risk"
        ],
        [
            "0_to_200",
            11,
            "credits_paid_to_date",
            "car_new",
            4553,
            "less_100",
            "less_1",
            3,
            "female",
            "none",
            3,
            "savings_insurance",
            22,
            "none",
            "own",
            1,
            "management_self-employed",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.637964656269084,
                0.362035343730916
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "no_checking",
            35,
            "outstanding_credit",
            "appliances",
            7138,
            "500_to_1000",
            "greater_7",
            5,
            "male",
            "co-applicant",
            4,
            "unknown",
            49,
            "none",
            "free",
            2,
            "skilled",
            2,
            "yes",
            "yes",
            "Risk",
            [
                0.11270206970758759,
                0.8872979302924124
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ],
        [
            "less_0",
            5,
            "all_credits_paid_back",
            "car_new",
            1523,
            "less_100",
            "unemployed",
            2,
            "female",
            "none",
            2,
            "real_estate",
            19,
            "none",
            "rent",
            1,
            "management_self-employed",
            1,
            "none",
            "yes",
            "No Risk",
            [
                0.7304597628653227,
                0.26954023713467745
            ],
            "Risk",
            [
                0.767955712021837,
                0.23204428797816307
            ],
            "Risk"
        ]
    ]
}

In [61]:
import urllib3, requests, json
def generate_access_token():
    headers={}
    headers["Content-Type"] = "application/x-www-form-urlencoded"
    headers["Accept"] = "application/json"
    auth = HTTPBasicAuth("bx", "bx")
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": CLOUD_API_KEY
    }
    response = requests.post(IAM_URL, data=data, headers=headers, auth=auth)
    json_data = response.json()
    iam_access_token = json_data['access_token']
    return iam_access_token

In [62]:
headers = {}
headers["Content-Type"] = "application/json"
headers["Authorization"] = "Bearer {}".format(generate_access_token())
WOS_GUID=data_mart_id

### Store the feedback payload using the data sets API

There are two ways OpenScale APIs can be used - a) using OpenScale Python SDK b) using OpenScale REST APIs.

For any reason if in the customer environment one cannot use the SDK, then the alternative is to use the REST APIs. The below cell demostrates to invoke one such OpenScale REST API, to log the feedback records to the OpenScale DataMart.

In [63]:
DATASETS_STORE_RECORDS_URL = "https://api.aiopenscale.cloud.ibm.com/openscale/{0}/v2/data_sets/{1}/records".format(data_mart_id, feedback_dataset_id)
for x in range(10):
    response = requests.post(DATASETS_STORE_RECORDS_URL, json=feedback_payload, headers=headers, verify=False)
    json_data = response.json()
    print(json_data)

{'state': 'preparing'}
{'state': 'preparing'}
{'state': 'preparing'}
{'state': 'preparing'}
{'state': 'preparing'}
{'state': 'preparing'}
{'state': 'preparing'}
{'state': 'preparing'}
{'state': 'preparing'}
{'state': 'preparing'}


### Wait for sometime, and make sure the records have reached to data sets related table.

In [64]:
time.sleep(10)
DATASETS_STORE_RECORDS_URL = "https://api.aiopenscale.cloud.ibm.com/openscale/{0}/v2/data_sets/{1}/records?limit={2}&include_total_count={3}".format(data_mart_id, feedback_dataset_id, 1, "true")
response = requests.get(DATASETS_STORE_RECORDS_URL, headers=headers, verify=False)
json_data = response.json()
print(json_data['total_count'])

100


## Run Quality Monitor

In [65]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run a6f74f22-8bd2-4b46-8c7a-65ae212d427a 




finished

---------------------------
 Successfully finished run 
---------------------------




In [66]:
run_details.to_dict()

{'metadata': {'id': 'a6f74f22-8bd2-4b46-8c7a-65ae212d427a',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:run:a6f74f22-8bd2-4b46-8c7a-65ae212d427a',
  'url': '/v2/monitor_instances/56e970b2-7bf2-405c-9f7b-2b026f6012ea/runs/a6f74f22-8bd2-4b46-8c7a-65ae212d427a',
  'created_at': '2024-08-08T03:38:33.965000Z',
  'created_by': 'IBMid-662005298W'},
 'entity': {'parameters': {'min_feedback_data_size': 50,
   'total_records_processed': 100},
  'status': {'state': 'finished',
   'queued_at': '2024-08-08T03:38:33.949000Z',
   'started_at': '2024-08-08T03:38:35.596000Z',
   'updated_at': '2024-08-08T03:38:38.220000Z',
   'completed_at': '2024-08-08T03:38:38.134000Z',
   'operators': [{'id': 'original',
     'status': {'state': 'finished',
      'started_at': '2024-08-08T03:38:34.173000Z',
      'completed_at': '2024-08-08T03:38:36.131000Z'},
     'result': {'confusion_matrix': {'feedback_records_filter_href': 'https:/

In [67]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2024-08-08 03:38:34.173000+00:00,true_positive_rate,75c0e16e-2c00-46f1-aa96-c62f914ccc4b,0.0,None,None,['model_type:original'],quality,56e970b2-7bf2-405c-9f7b-2b026f6012ea,a6f74f22-8bd2-4b46-8c7a-65ae212d427a,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:38:34.173000+00:00,area_under_roc,75c0e16e-2c00-46f1-aa96-c62f914ccc4b,0.5,0.8,None,['model_type:original'],quality,56e970b2-7bf2-405c-9f7b-2b026f6012ea,a6f74f22-8bd2-4b46-8c7a-65ae212d427a,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:38:34.173000+00:00,precision,75c0e16e-2c00-46f1-aa96-c62f914ccc4b,0.0,None,None,['model_type:original'],quality,56e970b2-7bf2-405c-9f7b-2b026f6012ea,a6f74f22-8bd2-4b46-8c7a-65ae212d427a,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:38:34.173000+00:00,matthews_correlation_coefficient,75c0e16e-2c00-46f1-aa96-c62f914ccc4b,0.0,None,None,['model_type:original'],quality,56e970b2-7bf2-405c-9f7b-2b026f6012ea,a6f74f22-8bd2-4b46-8c7a-65ae212d427a,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:38:34.173000+00:00,f1_measure,75c0e16e-2c00-46f1-aa96-c62f914ccc4b,0.0,None,None,['model_type:original'],quality,56e970b2-7bf2-405c-9f7b-2b026f6012ea,a6f74f22-8bd2-4b46-8c7a-65ae212d427a,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:38:34.173000+00:00,accuracy,75c0e16e-2c00-46f1-aa96-c62f914ccc4b,0.3,None,None,['model_type:original'],quality,56e970b2-7bf2-405c-9f7b-2b026f6012ea,a6f74f22-8bd2-4b46-8c7a-65ae212d427a,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:38:34.173000+00:00,label_skew,75c0e16e-2c00-46f1-aa96-c62f914ccc4b,0.8728715609439702,None,None,['model_type:original'],quality,56e970b2-7bf2-405c-9f7b-2b026f6012ea,a6f74f22-8bd2-4b46-8c7a-65ae212d427a,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:38:34.173000+00:00,gini_coefficient,75c0e16e-2c00-46f1-aa96-c62f914ccc4b,0.0,None,None,['model_type:original'],quality,56e970b2-7bf2-405c-9f7b-2b026f6012ea,a6f74f22-8bd2-4b46-8c7a-65ae212d427a,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:38:34.173000+00:00,false_positive_rate,75c0e16e-2c00-46f1-aa96-c62f914ccc4b,0.0,None,None,['model_type:original'],quality,56e970b2-7bf2-405c-9f7b-2b026f6012ea,a6f74f22-8bd2-4b46-8c7a-65ae212d427a,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:38:34.173000+00:00,area_under_pr,75c0e16e-2c00-46f1-aa96-c62f914ccc4b,0.7,None,None,['model_type:original'],quality,56e970b2-7bf2-405c-9f7b-2b026f6012ea,a6f74f22-8bd2-4b46-8c7a-65ae212d427a,subscription,fdaead88-3199-4c53-8a7f-85117919f62c


Note: First 10 records were displayed.


# Drift Configuration

## Upload the drift detection model archive generated as part of the common configuration archive

In [68]:
# Upload the drift archive to the subscription
'''from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

wslib.download_file("drift_v2_archive.tar.gz")'''

wos_client.monitor_instances.upload_drift_v2_archive(
    archive_path="drift_v2_archive.tar.gz",
    subscription_id=subscription_id
).result

{}

### Enable the drift monitor

In the following code cell, type a path to the drift configuration tar ball.

In the following code cell, default values are set for the drift monitor. You can change the default values by updating the values in the parameters section. The min_samples parameter controls the number of records that triggers the drift monitor to run. The drift_threshold parameter sets the threshold in decimal format for the drift percentage to trigger an alert. The train_drift_model parameter controls whether to re-train the model based on the drift analysis.

In [69]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
        "min_samples": 10,
        "max_samples": 1000,
        "train_archive": False
    }

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
print(drift_monitor_details)

{
  "metadata": {
    "id": "e86035af-ee95-4040-ba2b-35b1ca227bea",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:monitor_instance:e86035af-ee95-4040-ba2b-35b1ca227bea",
    "url": "/v2/monitor_instances/e86035af-ee95-4040-ba2b-35b1ca227bea",
    "created_at": "2024-08-08T03:41:14.084000Z",
    "created_by": "IBMid-662005298W"
  },
  "entity": {
    "data_mart_id": "80e6093f-5acf-4eb7-9da6-7ba9bf56a929",
    "monitor_definition_id": "drift_v2",
    "target": {
      "target_type": "subscription",
      "target_id": "fdaead88-3199-4c53-8a7f-85117919f62c"
    },
    "parameters": {
      "max_samples": 1000,
      "min_samples": 10,
      "train_archive": false
    },
    "thresholds": [
      {
        "metric_id": "confidence_drift_score",
        "type": "upper_limit",
        "value": 0.05
      },
      {
        "metric_id": "prediction_drift_score",
        "type": "upper_limit",
        "value": 0.05

## Check monitor instance status

In [70]:
drift_status = None

while drift_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
    drift_status = monitor_instance_details.entity.status.state
    if drift_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)

03:41:17 preparing
03:41:49 active


In [71]:
monitor_instance_details.to_dict()

{'metadata': {'id': 'e86035af-ee95-4040-ba2b-35b1ca227bea',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:monitor_instance:e86035af-ee95-4040-ba2b-35b1ca227bea',
  'url': '/v2/monitor_instances/e86035af-ee95-4040-ba2b-35b1ca227bea',
  'created_at': '2024-08-08T03:41:14.084000Z',
  'created_by': 'IBMid-662005298W',
  'modified_at': '2024-08-08T03:41:39.851000Z',
  'modified_by': 'iam-ServiceId-aeb7c04d-ba53-4adc-8008-8896ff6b462e'},
 'entity': {'data_mart_id': '80e6093f-5acf-4eb7-9da6-7ba9bf56a929',
  'monitor_definition_id': 'drift_v2',
  'target': {'target_type': 'subscription',
   'target_id': 'fdaead88-3199-4c53-8a7f-85117919f62c'},
  'parameters': {'advanced_controls': {},
   'context': {'baseline': {'data_set_id': 'a5449679-fdcd-4a12-8e6b-720ce90c330f'},
    'copy_subscription': {'enabled': False, 'source_id': None},
    'data_sets': {'drift_insights': 'd324c2a0-b12d-4630-90f8-df57b3f69e68',
     'drift

## Run an on-demand evaluation

In [72]:
# Check Drift monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "e86035af-ee95-4040-ba2b-35b1ca227bea",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:monitor_instance:e86035af-ee95-4040-ba2b-35b1ca227bea",
    "url": "/v2/monitor_instances/e86035af-ee95-4040-ba2b-35b1ca227bea",
    "created_at": "2024-08-08T03:41:14.084000Z",
    "created_by": "IBMid-662005298W",
    "modified_at": "2024-08-08T03:41:39.851000Z",
    "modified_by": "iam-ServiceId-aeb7c04d-ba53-4adc-8008-8896ff6b462e"
  },
  "entity": {
    "data_mart_id": "80e6093f-5acf-4eb7-9da6-7ba9bf56a929",
    "monitor_definition_id": "drift_v2",
    "target": {
      "target_type": "subscription",
      "target_id": "fdaead88-3199-4c53-8a7f-85117919f62c"
    },
    "parameters": {
      "advanced_controls": {},
      "context": {
        "baseline": {
          "data_set_id": "a5449679-fdcd-4a12-8e6b-720ce90c330f"
        },
        "copy_subscription": {
          "enabled": false,
   

In [73]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "a7ae12f6-8a58-418a-b25c-dd9c5f32c9f5",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/0a3c25959fab4ecea2768fa6b8d61595:80e6093f-5acf-4eb7-9da6-7ba9bf56a929:run:a7ae12f6-8a58-418a-b25c-dd9c5f32c9f5",
    "url": "/v2/monitor_instances/e86035af-ee95-4040-ba2b-35b1ca227bea/runs/a7ae12f6-8a58-418a-b25c-dd9c5f32c9f5",
    "created_at": "2024-08-08T03:42:15.392000Z",
    "created_by": "IBMid-662005298W"
  },
  "entity": {
    "parameters": {
      "advanced_controls": {},
      "context": {
        "baseline": {
          "data_set_id": "a5449679-fdcd-4a12-8e6b-720ce90c330f"
        },
        "copy_subscription": {
          "enabled": false,
          "source_id": null
        },
        "data_sets": {
          "drift_insights": "d324c2a0-b12d-4630-90f8-df57b3f69e68",
          "drift_intervals": "0eda6066-afae-461a-8358-702d58038068",
          "drift_scores": "d607971e-b3f7-4865-9e21-2fe76043b3f7",
          "drift_stats": "458c3579-9d41-4c13-923c-2

In [74]:
# Check run status

drift_run_status = None
while drift_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_run_status = monitoring_run_details.entity.status.state
    if drift_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), drift_run_status)
        time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), drift_run_status)

03:42:20 running
03:42:51 running
03:43:22 running
03:43:53 running
03:44:23 running
03:44:54 running
03:45:26 running
03:45:57 running
03:46:27 running
03:46:58 running
03:47:30 running
03:48:00 running
03:48:31 running
03:49:02 running
03:49:32 running
03:50:03 running
03:50:33 running
03:51:04 running
03:51:35 running
03:52:05 running
03:52:35 running
03:53:06 running
03:53:37 finished


## Display drift metrics

In [75]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2024-08-08 03:42:46.796668+00:00,records_processed,a12812c4-f82a-4334-b483-7a929e46557e,100.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,e86035af-ee95-4040-ba2b-35b1ca227bea,a7ae12f6-8a58-418a-b25c-dd9c5f32c9f5,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:42:46.796668+00:00,confidence_drift_score,a12812c4-f82a-4334-b483-7a929e46557e,0.5379,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,e86035af-ee95-4040-ba2b-35b1ca227bea,a7ae12f6-8a58-418a-b25c-dd9c5f32c9f5,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:42:46.796668+00:00,records_processed,a12812c4-f82a-4334-b483-7a929e46557e,100.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,e86035af-ee95-4040-ba2b-35b1ca227bea,a7ae12f6-8a58-418a-b25c-dd9c5f32c9f5,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:42:46.796668+00:00,confidence_drift_score,a12812c4-f82a-4334-b483-7a929e46557e,0.5378,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,e86035af-ee95-4040-ba2b-35b1ca227bea,a7ae12f6-8a58-418a-b25c-dd9c5f32c9f5,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:43:05.517049+00:00,records_processed,2068b7b3-d18c-4a74-becb-f93953bb9a71,100.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,e86035af-ee95-4040-ba2b-35b1ca227bea,a7ae12f6-8a58-418a-b25c-dd9c5f32c9f5,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:43:05.517049+00:00,confidence_drift_score,2068b7b3-d18c-4a74-becb-f93953bb9a71,0.5625,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,e86035af-ee95-4040-ba2b-35b1ca227bea,a7ae12f6-8a58-418a-b25c-dd9c5f32c9f5,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:43:05.517049+00:00,records_processed,2068b7b3-d18c-4a74-becb-f93953bb9a71,100.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,e86035af-ee95-4040-ba2b-35b1ca227bea,a7ae12f6-8a58-418a-b25c-dd9c5f32c9f5,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:43:05.517049+00:00,confidence_drift_score,2068b7b3-d18c-4a74-becb-f93953bb9a71,0.5623,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,e86035af-ee95-4040-ba2b-35b1ca227bea,a7ae12f6-8a58-418a-b25c-dd9c5f32c9f5,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:43:13.089346+00:00,records_processed,117e65d6-24c2-4df9-968a-82c645ff93aa,100.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:predictedLabel']",drift_v2,e86035af-ee95-4040-ba2b-35b1ca227bea,a7ae12f6-8a58-418a-b25c-dd9c5f32c9f5,subscription,fdaead88-3199-4c53-8a7f-85117919f62c
2024-08-08 03:43:13.089346+00:00,prediction_drift_score,117e65d6-24c2-4df9-968a-82c645ff93aa,1.0,None,0.05,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:predictedLabel']",drift_v2,e86035af-ee95-4040-ba2b-35b1ca227bea,a7ae12f6-8a58-418a-b25c-dd9c5f32c9f5,subscription,fdaead88-3199-4c53-8a7f-85117919f62c


Note: First 10 records were displayed.


## Cleanup the untarred files

In [ ]:
import os
files = ["common_configuration.json", "explainability.tar.gz", "drift_v2_archive.tar.gz"]
for file in files:
    if os.path.isfile(file):
        os.remove(file)

Author: Ravi Chamarthy (ravi.chamarthy@in.ibm.com)